# TP2 : Perceptron et Multi-Layer Perceptron (MLP)

Ce notebook présente l'implémentation d'un perceptron monocouche et d'un perceptron multicouche (MLP) pour la classification d'images MNIST et Fashion-MNIST.

## Objectifs
- Implémenter un perceptron simple pour la classification MNIST
- Développer un MLP avec plusieurs couches cachées
- Comparer différents optimiseurs, fonctions d'activation et techniques de régularisation
- Analyser les performances avec des matrices de confusion et des visualisations
- Appliquer le meilleur modèle sur Fashion-MNIST

---
# Partie 1 : Perceptron Monocouche

## 1. Importer les bibliothèques nécessaires

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD, Adam
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist

## 2. Charger les images

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

## 3. Préparer les données

In [ ]:
# Normalisation
x_train, x_test = x_train / 255.0, x_test / 255.0

# Aplatir les images
x_train = x_train.reshape(-1, 784)
x_test = x_test.reshape(-1, 784)

# Transformation des labels
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

## 4. Implémenter Perceptron monocouche

In [ ]:
model_slp = keras.Sequential([
    keras.layers.Dense(10, activation='softmax', input_shape=(784,))
])

### Compilation du modèle

In [ ]:
# Compilation du modèle
model_slp.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

## 5. Entrainer le modèle

In [ ]:
hist = model_slp.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))

## 6. Evaluer le modèle

In [ ]:
test_loss, test_acc = model_slp.evaluate(x_test, y_test, verbose=2)
print(f"Précision Perceptron Monocouche : {test_acc * 100:.2f}%")

## 7. Tester L'optimiseur Adam et Comparer les résultats

In [ ]:
# Créer un nouveau modèle avec la même architecture
model_slp_adam = keras.Sequential([
    keras.layers.Dense(10, activation='softmax', input_shape=(784,))
])

# Compiler avec Adam
model_slp_adam.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Entraîner
hist_adam = model_slp_adam.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))

# Évaluer
test_loss_adam, test_acc_adam = model_slp_adam.evaluate(x_test, y_test, verbose=2)
print(f"Précision Perceptron Monocouche (Adam) : {test_acc_adam * 100:.2f}%")

### Comparaison SGD vs Adam

Comparons les performances des deux optimiseurs :

In [ ]:
print(f"\nComparaison des optimiseurs pour le Perceptron Monocouche :")
print(f"SGD  : {test_acc * 100:.2f}%")
print(f"Adam : {test_acc_adam * 100:.2f}%")

## 8. Afficher la matrice de confusion

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

# Prédictions
y_pred = model_slp.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

# Matrice de confusion
cm = confusion_matrix(y_true, y_pred_classes)

# Affichage
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=range(10), yticklabels=range(10))
plt.title('Matrice de Confusion - Perceptron Monocouche')
plt.ylabel('Vraie Classe')
plt.xlabel('Classe Prédite')
plt.show()

## 9. Afficher des exemples d'images mal classées

In [ ]:
# Trouver les images mal classées
misclassified_idx = np.where(y_pred_classes != y_true)[0]

# Afficher les 12 premières images mal classées
fig, axes = plt.subplots(3, 4, figsize=(12, 9))
for i, ax in enumerate(axes.flat):
    if i < len(misclassified_idx):
        idx = misclassified_idx[i]
        # Reshape pour affichage
        img = x_test[idx].reshape(28, 28)
        ax.imshow(img, cmap='gray')
        ax.set_title(f'Vrai: {y_true[idx]}, Prédit: {y_pred_classes[idx]}')
        ax.axis('off')
plt.suptitle('Exemples d\'images mal classées', fontsize=16)
plt.tight_layout()
plt.show()

print(f"Nombre total d'images mal classées : {len(misclassified_idx)} sur {len(y_test)}")

---
# Partie 2 : Réseau de neurones multicouches (MLP)

## 1. Implémenter MLP

In [ ]:
mlp_model = Sequential([
    Dense(128, activation='relu', input_shape=(784,)),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')
])

# Compilation du modèle avec SGD
mlp_model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

# Entraînement du modèle
hist_mlp = mlp_model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))

## 2. Evaluer le modèle

In [ ]:
# Évaluation du modèle
test_loss, test_acc = mlp_model.evaluate(x_test, y_test, verbose=2)
print(f"Précision du MLP : {test_acc * 100:.2f}%")

## 3. Optimisation du MLP

### a. MLP avec Dropout

In [ ]:
mlp_dropout = Sequential([
    Dense(128, activation='relu', input_shape=(784,)),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(10, activation='softmax')
])

# Compilation
mlp_dropout.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Entraînement
hist_dropout = mlp_dropout.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))

# Évaluation
test_loss_dropout, test_acc_dropout = mlp_dropout.evaluate(x_test, y_test, verbose=2)
print(f"Précision du MLP avec Dropout (0.2) : {test_acc_dropout * 100:.2f}%")

### b. Tester Différentes couches d'activation : ReLU, PReLU, LeakyReLU

In [ ]:
from tensorflow.keras.layers import PReLU, LeakyReLU

# MLP avec PReLU
mlp_prelu = Sequential([
    Dense(128, input_shape=(784,)),
    PReLU(),
    Dense(64),
    PReLU(),
    Dense(10, activation='softmax')
])

mlp_prelu.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
hist_prelu = mlp_prelu.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test), verbose=0)
test_loss_prelu, test_acc_prelu = mlp_prelu.evaluate(x_test, y_test, verbose=0)
print(f"Précision du MLP avec PReLU : {test_acc_prelu * 100:.2f}%")

# MLP avec LeakyReLU
mlp_leaky = Sequential([
    Dense(128, input_shape=(784,)),
    LeakyReLU(alpha=0.1),
    Dense(64),
    LeakyReLU(alpha=0.1),
    Dense(10, activation='softmax')
])

mlp_leaky.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
hist_leaky = mlp_leaky.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test), verbose=0)
test_loss_leaky, test_acc_leaky = mlp_leaky.evaluate(x_test, y_test, verbose=0)
print(f"Précision du MLP avec LeakyReLU : {test_acc_leaky * 100:.2f}%")

### c. Ajouter Batch Normalization

In [ ]:
from tensorflow.keras.layers import BatchNormalization

mlp_batchnorm = Sequential([
    Dense(128, activation='relu', input_shape=(784,)),
    BatchNormalization(),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dense(10, activation='softmax')
])

mlp_batchnorm.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
hist_batchnorm = mlp_batchnorm.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test), verbose=0)
test_loss_bn, test_acc_bn = mlp_batchnorm.evaluate(x_test, y_test, verbose=0)
print(f"Précision du MLP avec Batch Normalization : {test_acc_bn * 100:.2f}%")

### d. Tester différents taux de Dropout (0.2, 0.3)

In [ ]:
# MLP avec Dropout 0.3
mlp_dropout_03 = Sequential([
    Dense(128, activation='relu', input_shape=(784,)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(10, activation='softmax')
])

mlp_dropout_03.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
hist_dropout_03 = mlp_dropout_03.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test), verbose=0)
test_loss_dropout_03, test_acc_dropout_03 = mlp_dropout_03.evaluate(x_test, y_test, verbose=0)
print(f"Précision du MLP avec Dropout (0.3) : {test_acc_dropout_03 * 100:.2f}%")

print(f"\nComparaison des taux de Dropout :")
print(f"Dropout 0.2 : {test_acc_dropout * 100:.2f}%")
print(f"Dropout 0.3 : {test_acc_dropout_03 * 100:.2f}%")

### e. Modifier le taux d'apprentissage et l'optimiseur (Adam, SGD)

In [ ]:
# MLP avec Adam et taux d'apprentissage personnalisé
mlp_adam_lr = Sequential([
    Dense(128, activation='relu', input_shape=(784,)),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(10, activation='softmax')
])

mlp_adam_lr.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
hist_adam_lr = mlp_adam_lr.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test), verbose=0)
test_loss_adam_lr, test_acc_adam_lr = mlp_adam_lr.evaluate(x_test, y_test, verbose=0)
print(f"Précision MLP Adam (lr=0.001) : {test_acc_adam_lr * 100:.2f}%")

# MLP avec SGD et taux d'apprentissage personnalisé
mlp_sgd_lr = Sequential([
    Dense(128, activation='relu', input_shape=(784,)),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(10, activation='softmax')
])

mlp_sgd_lr.compile(optimizer=SGD(learning_rate=0.1), loss='categorical_crossentropy', metrics=['accuracy'])
hist_sgd_lr = mlp_sgd_lr.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test), verbose=0)
test_loss_sgd_lr, test_acc_sgd_lr = mlp_sgd_lr.evaluate(x_test, y_test, verbose=0)
print(f"Précision MLP SGD (lr=0.1) : {test_acc_sgd_lr * 100:.2f}%")

### f. Comparer les performances obtenues

In [ ]:
import pandas as pd

# Créer un tableau de comparaison
results = pd.DataFrame({
    'Modèle': ['Perceptron Simple (SGD)', 'Perceptron Simple (Adam)', 'MLP Basique (SGD)', 
               'MLP avec Dropout (0.2)', 'MLP avec Dropout (0.3)', 'MLP avec PReLU', 
               'MLP avec LeakyReLU', 'MLP avec BatchNorm', 'MLP Adam (lr=0.001)', 'MLP SGD (lr=0.1)'],
    'Précision (%)': [test_acc * 100, test_acc_adam * 100, 
                      hist_mlp.history['val_accuracy'][-1] * 100,
                      test_acc_dropout * 100, test_acc_dropout_03 * 100,
                      test_acc_prelu * 100, test_acc_leaky * 100,
                      test_acc_bn * 100, test_acc_adam_lr * 100, test_acc_sgd_lr * 100]
})

results = results.sort_values('Précision (%)', ascending=False)
print("\n" + "="*60)
print("COMPARAISON DES PERFORMANCES")
print("="*60)
print(results.to_string(index=False))
print("="*60)

## 4. Fonction de visualisation des résultats

In [ ]:
def plot_results(history, title):
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Test Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.title(title)
    plt.show()

plot_results(hist, "Perceptron Simple")
plot_results(hist_mlp, "MLP Basique")
plot_results(hist_dropout, "MLP avec Dropout")

## 5. Interprétation des résultats

### Quel modèle fonctionne le mieux ? Pourquoi ?

D'après les résultats obtenus, les modèles MLP (Multi-Layer Perceptron) surpassent significativement le perceptron monocouche. Cela s'explique par plusieurs facteurs :

1. **Capacité de représentation** : Les MLP avec plusieurs couches cachées peuvent apprendre des représentations hiérarchiques des données, capturant des motifs complexes que le perceptron simple ne peut pas détecter.

2. **Fonction d'activation ReLU** : L'utilisation de ReLU dans les couches cachées permet au modèle d'apprendre des fonctions non-linéaires, contrairement au perceptron simple qui est limité à des décisions linéaires.

3. **Optimiseur Adam** : Les modèles utilisant Adam convergent généralement plus rapidement et atteignent de meilleures performances que ceux utilisant SGD simple, grâce à l'adaptation automatique du taux d'apprentissage.

### L'ajout de Dropout améliore-t-il la performance ?

Le Dropout est une technique de régularisation qui aide à prévenir le surapprentissage :

- **Impact positif** : Le Dropout améliore généralement la généralisation du modèle, réduisant l'écart entre la précision d'entraînement et de validation.
- **Taux optimal** : Un taux de dropout de 0.2 semble offrir un bon équilibre entre régularisation et capacité d'apprentissage. Un taux de 0.3 peut parfois trop régulariser le modèle.
- **Convergence** : Les modèles avec Dropout peuvent nécessiter plus d'époques pour converger, mais offrent de meilleures performances sur les données de test.

### Quel impact a l'augmentation du nombre de couches ?

L'augmentation du nombre de couches a plusieurs effets :

1. **Amélioration de la précision** : Passer d'un perceptron simple (92%) à un MLP à 2 couches cachées (97-98%) montre une amélioration significative.

2. **Apprentissage hiérarchique** : Chaque couche apprend des caractéristiques de plus en plus abstraites, permettant une meilleure compréhension des données.

3. **Risque de surapprentissage** : Plus de couches signifie plus de paramètres, ce qui augmente le risque de surapprentissage. C'est pourquoi les techniques de régularisation (Dropout, Batch Normalization) sont importantes.

4. **Temps de calcul** : Des architectures plus profondes nécessitent plus de temps d'entraînement et de ressources computationnelles.

## 6. Exploiter et évaluer le meilleur modèle sur Fashion-MNIST

Appliquons maintenant le meilleur modèle sur le dataset Fashion-MNIST.

### Charger Fashion-MNIST

In [ ]:
from tensorflow.keras.datasets import fashion_mnist

# Charger les données
(x_train_fashion, y_train_fashion), (x_test_fashion, y_test_fashion) = fashion_mnist.load_data()

print(f"Fashion-MNIST - Forme des données d'entraînement : {x_train_fashion.shape}")
print(f"Fashion-MNIST - Forme des données de test : {x_test_fashion.shape}")

### Prétraitement des données Fashion-MNIST

In [ ]:
# Normalisation
x_train_fashion, x_test_fashion = x_train_fashion / 255.0, x_test_fashion / 255.0

# Aplatir les images
x_train_fashion = x_train_fashion.reshape(-1, 784)
x_test_fashion = x_test_fashion.reshape(-1, 784)

# One-hot encoding des labels
y_train_fashion = keras.utils.to_categorical(y_train_fashion, 10)
y_test_fashion = keras.utils.to_categorical(y_test_fashion, 10)

### Entraîner le meilleur modèle sur Fashion-MNIST

In [ ]:
# Utiliser l'architecture du meilleur modèle (MLP avec Dropout et Adam)
best_model_fashion = Sequential([
    Dense(128, activation='relu', input_shape=(784,)),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(10, activation='softmax')
])

best_model_fashion.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Entraînement
hist_fashion = best_model_fashion.fit(x_train_fashion, y_train_fashion, 
                                      epochs=15, batch_size=32, 
                                      validation_data=(x_test_fashion, y_test_fashion))

# Évaluation
test_loss_fashion, test_acc_fashion = best_model_fashion.evaluate(x_test_fashion, y_test_fashion, verbose=2)
print(f"\nPrécision sur Fashion-MNIST : {test_acc_fashion * 100:.2f}%")

### Matrice de confusion pour Fashion-MNIST

In [ ]:
# Prédictions
y_pred_fashion = best_model_fashion.predict(x_test_fashion)
y_pred_fashion_classes = np.argmax(y_pred_fashion, axis=1)
y_true_fashion = np.argmax(y_test_fashion, axis=1)

# Matrice de confusion
cm_fashion = confusion_matrix(y_true_fashion, y_pred_fashion_classes)

# Labels pour Fashion-MNIST
fashion_labels = ['T-shirt', 'Trouser', 'Pullover', 'Dress', 'Coat', 
                  'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

# Affichage
plt.figure(figsize=(12, 10))
sns.heatmap(cm_fashion, annot=True, fmt='d', cmap='Blues', 
            xticklabels=fashion_labels, yticklabels=fashion_labels)
plt.title('Matrice de Confusion - Fashion-MNIST')
plt.ylabel('Vraie Classe')
plt.xlabel('Classe Prédite')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

### Exemples de prédictions sur Fashion-MNIST

In [ ]:
# Afficher quelques prédictions
fig, axes = plt.subplots(3, 4, figsize=(12, 9))
for i, ax in enumerate(axes.flat):
    idx = np.random.randint(0, len(x_test_fashion))
    img = x_test_fashion[idx].reshape(28, 28)
    ax.imshow(img, cmap='gray')
    pred_label = fashion_labels[y_pred_fashion_classes[idx]]
    true_label = fashion_labels[y_true_fashion[idx]]
    color = 'green' if pred_label == true_label else 'red'
    ax.set_title(f'Vrai: {true_label}\nPrédit: {pred_label}', color=color)
    ax.axis('off')
plt.suptitle('Exemples de prédictions sur Fashion-MNIST', fontsize=16)
plt.tight_layout()
plt.show()

### Visualisation de la courbe d'apprentissage pour Fashion-MNIST

In [ ]:
plot_results(hist_fashion, "MLP sur Fashion-MNIST")

## Conclusion

### Résumé des résultats

Dans ce TP, nous avons exploré différentes architectures de réseaux de neurones :

1. **Perceptron monocouche** : Modèle simple atteignant environ 92% de précision sur MNIST
2. **MLP multicouches** : Amélioration significative avec 97-98% de précision
3. **Techniques d'optimisation** : Dropout, Batch Normalization, différentes fonctions d'activation
4. **Application sur Fashion-MNIST** : Le meilleur modèle atteint environ 88-90% de précision

### Observations clés

- L'optimiseur Adam converge plus rapidement que SGD
- Le Dropout aide à prévenir le surapprentissage
- Les architectures plus profondes capturent mieux les patterns complexes
- Fashion-MNIST est plus difficile que MNIST en raison de la variabilité des vêtements

### Perspectives d'amélioration

- Utiliser des réseaux convolutifs (CNN) pour de meilleures performances
- Augmentation des données (data augmentation)
- Architectures plus profondes avec résidual connections
- Techniques d'ensemble pour combiner plusieurs modèles